# Tarea 2

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

# from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

## Funciones necesarias para las que se crearán

Importación de los datos de la curva cero cupón

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [3]:
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
curva.head()

,plazo,tasa,df,t,rate
0,1,0.000811,0.999998,0.002740,0.000811
1,7,0.000841,0.999984,0.019178,0.000841
2,14,0.000780,0.999970,0.038356,0.000780
3,21,0.000774,0.999955,0.057534,0.000774
4,33,0.000781,0.999929,0.090411,0.000781


Obtención de las tasas de la curva mediante interpolación

In [4]:
zcurva = interp1d(curva['t'], curva['rate'], kind='cubic',
                  fill_value="extrapolate")

### Construcción de Theta

Función $\theta_{t}$:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$,

y también que:

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

In [5]:
#Funciones para las derivadas de la tasa cero cupón

def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

Funciones para la tasa forward y su derivada:

In [6]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [7]:
#Valores arbitarios para gamma y sigma:

sigma=0.015
gamma=0.5

#valor estimado de r0:
r0 = zrate(0)    

In [8]:
#Función para obtener el valor de theta

def theta(t, gamma, sigma):
    theta= dfwd(t) + gamma*fwd(t) + ((sigma**2)/(2*gamma) * (1- math.exp(-2*gamma*t)))
    return theta

In [9]:
#Solo para probar

theta_prueba= theta(2,gamma, sigma)

print(f"Valor de Theta cuando t=2 : {theta_prueba: .8%}")

Valor de Theta cuando t=2 :  0.14487625%


Función para las simulaciones de $r_t$

In [10]:
#función para las simulaciones

def simulaciones_HW(gamma, sigma, r0, num_sim, num_steps, seed=None): 
    """
    Calcula la tasa libre de riesgo r mediante el modelo de Hull y White para num_sim cantidad de simulaciones, para
    distintos steps de tiempo determinados por num_steps
    """
    
    dt=1/264
    num_steps= num_steps+1
    
    #cálculo de los números aleatorios
    dXt = np.zeros((num_sim, num_steps))
    np.random.seed(seed)
    
    for i in range(0, num_sim):
        for j in range(0, num_steps):
            dXt[i,j] = np.random.normal()     #generación de la matriz de números aleatorios
            
    #cálculo de los Theta (son determinísticos, no dependen de la simulación)
    thetas=np.zeros((1,num_steps))      #matriz de thetas para cada t
    tiempo = np.zeros((1,num_steps))    #matriz con los steps del tiempo
    
    for i in range(0,num_steps):
        tiempo[0,i]= i * dt
        thetas[0,i]=theta(i*dt, gamma, sigma) 
    
    #Simulación de las trayectorias para r
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma*dt
    simulaciones = np.zeros((num_sim, num_steps))
    
    simulaciones[:,0] = r0         #valor inicial de r para cada simulación
    
    for i in range(0, num_sim):
        r=r0        #cada simulación inicia con r=r0
        
        for j in range(1,num_steps):
            r = r + thetas[0,(j-1)] * dt - gamma_dt * r + sqdt_sigma * dXt[i,(j-1)]
            simulaciones[i,j] = r
            
    return tiempo, simulaciones    #retorna el valor de r para cada step en cada simulación
    

In [11]:
#PRUEBAS

num_sim=10
num_steps=263

#verificando que el primer valor de cada simulación sea r0
prueba_tiempo,prueba_sims=simulaciones_HW(gamma, sigma, r0, num_sim, num_steps, 1000)
#prueba_sims[4,:]

r_final= prueba_sims[0,10]
print(f"Con una semilla predeterminada de 1000, el valor de la tasa en la última simulación de la prueba es: {r_final: .8}")

Con una semilla predeterminada de 1000, el valor de la tasa en la última simulación de la prueba es:  0.00050987803


Función para el cálculo del valor del bono cero cupóns, no se utiliza nunca, pero se dejó porque sí

In [12]:
#cálculo de B
def get_B_HW(t,T,gamma):
    B = (1/gamma) * (1- math.exp(-gamma*(T-t)))
    
    return B
    

In [13]:
#cálculo de A
def get_A_HW(t, T, gamma, sigma):
    B= get_B_HW(t,T,gamma)
    dfT=math.exp(-zrate(T)*T)
    dft=math.exp(-zrate(t)*t)
    A=math.log(dfT/dft) + B*fwd(t) - ((sigma**2)/(4*gamma) * B**2 * (1- math.exp(-2*gamma*t)))
    
    return A

In [14]:
#cálculo bono cero cupón
def zero_HW(r, t, T, gamma, sigma):
    A = get_A_HW(t, T, gamma, sigma)
    B = get_B_HW(t, T, gamma)
    
    bono_zc = math.exp(A - B * r)
    
    return bono_zc

# Función para Calls y Puts con MC

In [15]:
# Función para las Calls y las Puts usando el método de MonteCarlo

def valor_opcion_MC(t, T, gamma, sigma, r0, K, num_sim, num_steps, opcion, seed=None):
    """
    Calcula el valor de una opción call o put sobre un bono cero cupón con el método de Hull y White. Se realizan 
    "num_sim" simulaciones en total para los parámetros dados

    """

    if t > T:
        print('La opción no tiene valor ya que se está después luego de su expiración')

    # simulaciones de las tasas con MC y HW con la función ya existente
    tiempo, tasas_r = simulaciones_HW(
        gamma, sigma, r0, num_sim, num_steps, seed)

    # se guarda el valor final de las tasas para cada simulación
    tasas_finales = tasas_r[:, num_steps]
    # matriz para guardar los discount factor (precio del bono)
    discount_factor = np.zeros((num_sim, 1))
    payoff = np.zeros((num_sim, 1))

    # Para cada simulación se calcula el payoff de la opción, para luego hacer un promedio de estos y
    # traerlo a valor presente. Calcula por lo tanto el valor de la opción para el caso de una call ó una put.

    for i in range(0, num_sim):
        # cálculo del precio del bono para cada r al final de cada simulación
        discount_factor[i, 0] = zero_HW(tasas_finales[i], t, T, gamma, sigma)

        if opcion == "call":
            # cálculo del payoff dependiendo del tipo de opción
            payoff[i, 0] = max((discount_factor[i, 0]-K), 0)

        elif opcion == "put":
            payoff[i, 0] = max((K-discount_factor[i, 0]), 0)

        else:
            print('La opción debe ser call o put')

    # cálculo del promedio de las simulaciones del precio de las opciones
    valor_opcion = np.average(payoff)

    valor_presente = math.exp(-t * float(zrate(t))) * \
        valor_opcion  # se trae a valor presente
    # AD: el valor presente está mal calculado. Debe ser con el factor de descuento de la trayectoria.

    return valor_presente

In [16]:
# Prueba del valor de la opción call con Monte carlo con parámetros arbitrarios, con 1000 simulaciones y semilla 1000.

t = 1
T = 2
K = 1
num_steps = 263
num_sim = 1000
sigma = 0.015
gamma = 0.5
r0 = zrate(1/264)

call_MC = valor_opcion_MC(t, T, gamma, sigma, r0, K,
                          num_sim, num_steps, "call", 1000)


print(
    f"El valor de la call calculado con Monte Carlo, con los parámetros establecidos es: {call_MC: .8}")

El valor de la call calculado con Monte Carlo, con los parámetros establecidos es:  0.0038614301


In [17]:
# Prueba del valor de la opción put con Monte carlo con parámetros arbitrarios con 1000 simulaciones y semilla 1000.


put_MC = valor_opcion_MC(t, T, gamma, sigma, r0, K,
                         num_sim, num_steps, "put", 1000)

print(
    f"El valor de la put calculado con Monte Carlo, con los parámetros anteriores es: {put_MC: .8}")

El valor de la put calculado con Monte Carlo, con los parámetros anteriores es:  0.0037686762


## Función para Calls y Puts con Fórmula Teórica

In [18]:
# Función para el d1

def get_d1(r0, TB, TO, gamma, sigma, K):
    """
    Se calcula el d1 para los parámetros dados, donde TB corresponde a la madurez del bono cero cupón y TO la madurez
    de la opción
    """

    dfTB = math.exp(-zrate(TB)*TB)  # fatores de descuento para TB y TO
    dfTO = math.exp(-zrate(TO)*TO)
    Sz = get_B_HW(TO, TB, gamma) * math.sqrt((sigma**2) /
                                             (2*gamma) * (1 - math.exp(-2*gamma*TO)))

    d1 = (math.log((1/K)*dfTB/dfTO) + 0.5 * (Sz**2)) / Sz

    return d1

In [19]:
# Función para el d2
def get_d2(r0, TB, TO, gamma, sigma, K):
    Sz = get_B_HW(TO, TB, gamma) * math.sqrt((sigma**2) /
                                             (2*gamma) * (1 - math.exp(-2*gamma*TO)))
    d1 = get_d1(r0, TB, TO, gamma, sigma, K)

    d2 = d1 - Sz

    return d2

In [20]:
# Función para el valor de la opción ORIGINAL

def valor_opcion_BS(r0, TB, TO, gamma, sigma, K, opcion):

    if TO > TB:
        print('La opción no tiene valor ya que la madurez de esta es mayor que la del bono subyacente')

    dfTB = math.exp(-zrate(TB)*TB)
    dfTO = math.exp(-zrate(TO)*TO)

    d1 = get_d1(r0, TB, TO, gamma, sigma, K)
    d2 = get_d2(r0, TB, TO, gamma, sigma, K)

    if opcion == "call":
        call = dfTO * ((dfTB/dfTO)*norm.cdf(d1) - K*norm.cdf(d2))
        valor_presente = call

    elif opcion == "put":
        put = dfTO * (K*norm.cdf(-d2) - (dfTB/dfTO)*norm.cdf(-d1))
        valor_presente = put

    else:
        print('La opción debe ser call o put')

    return valor_presente

In [21]:
# Prueba del valor de la opción call con Black Scholes con parámetros arbitrarios.

opcion = "call"
r0 = r0
t = 0
TB = 2
TO = 1
K = 1

call_BS = valor_opcion_BS(r0, TB, TO, gamma, sigma, K, opcion)

print(
    f"El valor de la call calculado con Black Scholes, con los parámetros anteriores es: {call_BS: .8}")

El valor de la call calculado con Black Scholes, con los parámetros anteriores es:  0.0035283883


In [34]:
import hull_white as hw

In [36]:
# Test

hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=K,
    to=TO,
    tb=TB,
    r0=r0,
    zo=math.exp(-zrate(TO) * TO),
    zb=math.exp(-zrate(TB) * TB),
    gamma=gamma,
    sigma=sigma)

0.0035283883401179272

In [37]:
# Prueba del valor de la opción put con Black Scholes con parámetros arbitrarios.


opcion2 = "put"

put_BS = valor_opcion_BS(r0, TB, TO, gamma, sigma, K, opcion2)

print(
    f"El valor de la put calculado con Black Scholes, con los parámetros anteriores es: {put_BS: .8}")

El valor de la put calculado con Black Scholes, con los parámetros anteriores es:  0.0039607699


In [38]:
# Test

hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=K,
    to=TO,
    tb=TB,
    r0=r0,
    zo=math.exp(-zrate(TO) * TO),
    zb=math.exp(-zrate(TB) * TB),
    gamma=gamma,
    sigma=sigma)

0.00396076991674188

In [23]:
# Error absoluto entre el valor de la call calculada con Monte carlo versus Black Scholes.

error_call = abs(call_MC-call_BS)

print(f"Valor call con MC: {call_MC: .8}")
print(f"Valor call con BS: {call_BS: .8}")


print(
    f"Usando 1000 simulaciones y una semilla predeterminada de 1000, el error de la call entre MC y las fórmulas del modelo es: {error_call: .8}")

Valor call con MC:  0.0038614301
Valor call con BS:  0.0035283883
Usando 1000 simulaciones y una semilla predeterminada de 1000, el error de la call entre MC y las fórmulas del modelo es:  0.00033304178


In [24]:
# Error absoluto entre el valor de la put calculada con Monte carlo versus Black Scholes.

error_put = abs(put_MC - put_BS)

print(f"Valor put con MC: {put_MC: .8}")
print(f"Valor put con BS: {put_BS: .8}")


print(
    f"Usando 1000 simulaciones y una semilla predeterminada de 1000, el error absoluto de la put entre MC y las fórmulas del modelo es: {error_put: .8}")

Valor put con MC:  0.0037686762
Valor put con BS:  0.0039607699
Usando 1000 simulaciones y una semilla predeterminada de 1000, el error absoluto de la put entre MC y las fórmulas del modelo es:  0.00019209369


In [25]:
# Cálculos para la opción call y call, para distintos strikes

opcion = "call"
r0 = r0
t = 1
t_BS = 0
TB = 2
TO = 1
strikes = [0.99, 0.98, 1, 1.01, 1.02]

valores_call_MC = np.zeros((1, len(strikes)))
valores_call_BS = np.zeros((1, len(strikes)))
valores_put_MC = np.zeros((1, len(strikes)))
valores_put_BS = np.zeros((1, len(strikes)))

errores_call = np.zeros((1, len(strikes)))
errores_put = np.zeros((1, len(strikes)))

for i in range(1, len(strikes)):
    valores_call_MC[0, i] = valor_opcion_MC(
        t, T, gamma, sigma, r0, strikes[i], num_sim, num_steps, "call", 1000)
    valores_put_MC[0, i] = valor_opcion_MC(
        t, T, gamma, sigma, r0, strikes[i], num_sim, num_steps, "put", 1000)

    valores_call_BS[0, i] = valor_opcion_BS(
        r0, TB, TO, gamma, sigma, strikes[i], "call")
    valores_put_BS[0, i] = valor_opcion_BS(
        r0, TB, TO, gamma, sigma, strikes[i], "put")

    errores_call[0, i] = abs(valores_call_MC[0, i] - valores_call_BS[0, i])
    errores_put[0, i] = abs(valores_put_MC[0, i] - valores_put_BS[0, i])

In [26]:
for i in range(0, len(strikes)):
    print(f'Strike K = : {strikes[i]:.2f}')
    print(f'Call con Monte Carlo: {valores_call_MC[0,i]:.8f}')
    print(f'Call con Black Scholes: {valores_call_BS[0,i]:.8f}')
    print(f'Error Absoluto: {errores_call[0,i]:.8}')
    print('')


error_promedio_call = np.average(errores_call)
print(f'Error Promedio para todos los strikes: {error_promedio_call:.8f}')

Strike K = : 0.99
Call con Monte Carlo: 0.00000000
Call con Black Scholes: 0.00000000
Error Absoluto: 0.0

Strike K = : 0.98
Call con Monte Carlo: 0.02014252
Call con Black Scholes: 0.01961254
Error Absoluto: 0.00052997986

Strike K = : 1.00
Call con Monte Carlo: 0.00386143
Call con Black Scholes: 0.00352839
Error Absoluto: 0.00033304178

Strike K = : 1.01
Call con Monte Carlo: 0.00065798
Call con Black Scholes: 0.00063852
Error Absoluto: 1.9460033e-05

Strike K = : 1.02
Call con Monte Carlo: 0.00003599
Call con Black Scholes: 0.00005238
Error Absoluto: 1.6398882e-05

Error Promedio para todos los strikes: 0.00017978


In [27]:
for i in range(0,len(strikes)):
    print(f'Strike K = : {strikes[i]:.2f}')
    print(f'Put con Monte Carlo: {valores_put_MC[0,i]:.8f}')
    print(f'Put con Black Scholes: {valores_put_BS[0,i]:.8f}')
    print(f'Error Absoluto: {errores_put[0,i]:.8}')
    print('')
    
error_promedio_put = np.average(errores_put)
print(f'Error Promedio para todos los strikes: {error_promedio_put:.8f}')

Strike K = : 0.99
Put con Monte Carlo: 0.00000000
Put con Black Scholes: 0.00000000
Error Absoluto: 0.0

Strike K = : 0.98
Put con Monte Carlo: 0.00006381
Put con Black Scholes: 0.00005896
Error Absoluto: 4.8443854e-06

Strike K = : 1.00
Put con Monte Carlo: 0.00376868
Put con Black Scholes: 0.00396077
Error Absoluto: 0.00019209369

Strike K = : 1.01
Put con Monte Carlo: 0.01055820
Put con Black Scholes: 0.01106388
Error Absoluto: 0.00050567544

Strike K = : 1.02
Put con Monte Carlo: 0.01992919
Put con Black Scholes: 0.02047072
Error Absoluto: 0.00054153436

Error Promedio para todos los strikes: 0.00024883


**AD:** bien, vieron el error para varios casos.

## Bono Cupón Cero a 1Y

In [28]:
#para saber el índice y el valor del df de la curva a 365 días (1Y)
df_1Y=curva.loc[curva['plazo'] == 365, 'df']
print(df_1Y)

15    0.999298
Name: df, dtype: float64


In [29]:
#Se obtiene el valor de mercado del bono directamente de la curva 
valor_mercado_bono = curva['df'].iloc[15]

print(f"El valor de mercado del bono a 1Y se sacó directamente de la curva y es el siguiente: {valor_mercado_bono: .8}")


El valor de mercado del bono a 1Y se sacó directamente de la curva y es el siguiente:  0.99929787


Cálculo del bono con MC

In [30]:
def valor_bono_MC(dt, gamma, sigma, r0, T_bono, num_sim, num_steps, seed=None):

    # Para valorizar el bono a 1Y, reutilizamos las simulaciones de las tasas que hicimos anteriormente, por lo que

    # se simulan nuevamente las tasas con MC y HW
    # se obtiene la discretización del tiempo y las simulaciones de las tasas
    tiempo, simulaciones_tasas = simulaciones_HW(
        gamma, sigma, r0, num_sim, num_steps, 1000)
    tasas_finales = simulaciones_tasas[:, num_steps]

    df = 0  # se crea la variable df

    for sim in simulaciones_tasas:
        df += math.exp(-dt * np.sum(sim))

    bono_MC = df/num_sim

    return bono_MC

In [31]:
# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(1/264)
T_bono = 1
dt = 1/264
num_sim = 1000
num_steps = 263

bono_MC = valor_bono_MC(dt, gamma, sigma, r0, T_bono, num_sim, num_steps, 1000)

print(
    f"El valor del bono mediante simulaciones de Monte Carlo es: {bono_MC: .8}")

El valor del bono mediante simulaciones de Monte Carlo es:  0.99969823


In [32]:
error_bono = abs(valor_mercado_bono - bono_MC)

print(f"Valor bono con MC: {bono_MC: .8}")
print(f"Valor bono de mercado: {valor_mercado_bono: .8}")

print(
    f"Usando 1000 simulaciones y una semilla predeterminada de 1000, el error absoluto del bono entre el calculado con MC y el de mercado es: {error_bono: .8}")

Valor bono con MC:  0.99969823
Valor bono de mercado:  0.99929787
Usando 1000 simulaciones y una semilla predeterminada de 1000, el error absoluto del bono entre el calculado con MC y el de mercado es:  0.00040036272


**AD:** hubiera sido excelente que también probaran factores de descuento a varios plazos.